In [ ]:
"""
Purpose: To decompose the multi-somas for splitting
using the new decomposition method



"""


In [1]:
%load_ext autoreload
%autoreload 2

In [2]:
import numpy as np
import datajoint as dj
import trimesh
from tqdm.notebook import tqdm
from pathlib import Path

from os import sys
sys.path.append("/meshAfterParty/")

import datajoint_utils as du
from importlib import reload

INFO - 2021-01-03 23:16:12,970 - settings - Setting database.host to at-database.ad.bcm.edu
INFO - 2021-01-03 23:16:12,971 - settings - Setting database.user to celiib
INFO - 2021-01-03 23:16:12,972 - settings - Setting database.password to newceliipass
INFO - 2021-01-03 23:16:12,976 - settings - Setting stores to {'minnie65': {'protocol': 'file', 'location': '/mnt/dj-stor01/platinum/minnie65', 'stage': '/mnt/dj-stor01/platinum/minnie65'}, 'meshes': {'protocol': 'file', 'location': '/mnt/dj-stor01/platinum/minnie65/02/meshes', 'stage': '/mnt/dj-stor01/platinum/minnie65/02/meshes'}, 'decimated_meshes': {'protocol': 'file', 'location': '/mnt/dj-stor01/platinum/minnie65/02/decimated_meshes', 'stage': '/mnt/dj-stor01/platinum/minnie65/02/decimated_meshes'}, 'skeletons': {'protocol': 'file', 'location': '/mnt/dj-stor01/platinum/minnie65/02/skeletons'}}
INFO - 2021-01-03 23:16:12,976 - settings - Setting enable_python_native_blobs to True
INFO - 2021-01-03 23:16:12,987 - connection - Connect

Connecting celiib@at-database.ad.bcm.edu:3306


INFO - 2021-01-03 23:16:13,221 - settings - Setting enable_python_native_blobs to True


In [3]:
test_mode = True

In [4]:
import minfig
import time
import numpy as np
#want to add in a wait for the connection part
random_sleep_sec = np.random.randint(0, 200)
print(f"Sleeping {random_sleep_sec} sec before conneting")
if not test_mode:
    time.sleep(random_sleep_sec)
print("Done sleeping")

du.config_celii()
du.set_minnie65_config_segmentation(minfig)
du.print_minnie65_config_paths(minfig)

#configuring will include the adapters
minnie,schema = du.configure_minnie_vm()

INFO - 2021-01-03 23:16:13,339 - settings - Setting database.host to at-database.ad.bcm.edu
INFO - 2021-01-03 23:16:13,340 - settings - Setting database.user to celiib
INFO - 2021-01-03 23:16:13,341 - settings - Setting database.password to newceliipass
INFO - 2021-01-03 23:16:13,342 - settings - Setting enable_python_native_blobs to True


Sleeping 94 sec before conneting
Done sleeping
Current path for external_segmentation_path = /mnt/dj-stor01/platinum/minnie65/02
Current path for external_mesh_path = /mnt/dj-stor01/platinum/minnie65/02/meshes
Current path for external_decimated_mesh_path = /mnt/dj-stor01/platinum/minnie65/02/decimated_meshes
Current path for external_skeleton_path = /mnt/dj-stor01/platinum/minnie65/02/skeletons


INFO - 2021-01-03 23:16:13,570 - settings - Setting enable_python_native_blobs to True


# Getting the list of neurons to decompose for the mutli soma testing

In [5]:
# import pandas as pd
# soma_soma_table = pd.read_csv("Minnie65 core proofreading - Soma-Soma.csv")
# no_header = soma_soma_table.iloc[1:]
# multi_soma_ids_str = no_header["Dendrites"].to_numpy()
# multi_soma_ids = multi_soma_ids_str[~np.isnan(multi_soma_ids_str.astype("float"))].astype("int")

# @schema
# class MultiSomaProofread(dj.Manual):
#     definition="""
#     segment_id : bigint unsigned  #segment id for those to be decimated
#     """
    
# dict_of_seg = [dict(segment_id=k) for k in multi_soma_ids]
# minnie.MultiSomaProofread.insert(dict_of_seg,skip_duplicates=True)
# MultiSomaProofread()

# Defining the Table

In [6]:
import neuron_utils as nru
import neuron
import trimesh_utils as tu
import numpy as np

In [7]:
import meshlab
meshlab.set_meshlab_port(current_port=None)

No port chosen so picked random port 120


In [8]:
#so that it will have the adapter defined
from datajoint_utils import *

In [9]:
decimation_version = 0
decimation_ratio = 0.25
key_source = (minnie.Decimation().proj(decimation_version='version')  & 
              dict(decimation_version=decimation_version,decimation_ratio=decimation_ratio)  
              & minnie.MultiSomaProofread() & (dj.U("segment_id") & (minnie.BaylorSegmentCentroid() & "multiplicity=2").proj()))
key_source

segment_id segment id unique within each Segmentation,decimation_version,decimation_ratio ratio of remaining mesh vertices/faces (which ones depends on what metric the decimation technique uses)
864691134988402042,0,0.25
864691134988472442,0,0.25
864691135012400886,0,0.25
864691135081644023,0,0.25
864691135081658359,0,0.25
864691135103989069,0,0.25
864691135181738242,0,0.25
864691135181779714,0,0.25
864691135207768697,0,0.25
864691135210033344,0,0.25


In [10]:
import numpy as np
import time
decimation_version = 0
decimation_ratio = 0.25

@schema
class Decomposition(dj.Computed):
    definition="""
    -> minnie.Decimation.proj(decimation_version='version')
    ---
    decomposition: <decomposition>
    n_vertices           : int unsigned                 # number of vertices
    n_faces              : int unsigned                 # number of faces
    n_error_limbs: int #the number of limbs that are touching multiple somas or 1 soma in multiple places
    n_same_soma_multi_touching_limbs: int # number of limbs that touch the same soma multiple times
    n_multi_soma_touching_limbs: int # number of limbs that touch multiple somas
    n_somas: int #number of soma meshes detected
    n_limbs: int
    n_branches: int
    max_limb_n_branches=NULL:int
    
    skeletal_length=NULL: double
    max_limb_skeletal_length=NULL:double
    median_branch_length=NULL:double #gives information on average skeletal length to next branch point
    
    
    width_median=NULL: double #median width from mesh center without spines removed
    width_no_spine_median=NULL: double #median width from mesh center with spines removed
    width_90_perc=NULL: double # 90th percentile for width without spines removed
    width_no_spine_90_perc=NULL: double  # 90th percentile for width with spines removed
    
    
    n_spines: bigint

    spine_density=NULL: double # n_spines/ skeletal_length
    spines_per_branch=NULL: double
    
    skeletal_length_eligible=NULL: double # the skeletal length for all branches searched for spines
    n_spine_eligible_branches=NULL: int # the number of branches that were checked for spines because passed width threshold
    
    spine_density_eligible=NULL:double # n_spines/skeletal_length_eligible
    spines_per_branch_eligible=NULL:double # n_spines/n_spine_eligible_branches
    
    total_spine_volume=NULL: double # the sum of all spine volume
    spine_volume_median=NULL: double # median of the spine volume for those spines with able to calculate volume
    spine_volume_density=NULL: double #total_spine_volume/skeletal_length
    spine_volume_density_eligible=NULL: double #total_spine_volume/skeletal_length_eligible
    spine_volume_per_branch_eligible=NULL: double #total_spine_volume/n_spine_eligible_branches
    
    run_time=NULL : double                   # the amount of time to run (seconds)

    
    """

#     key_source =  ((minnie.Decimation).proj(decimation_version='version') & 
#                             "decimation_version=" + str(decimation_version) &
#                        f"decimation_ratio={decimation_ratio}" &  (minnie.BaylorSegmentCentroid() & "multiplicity>0" & "segment_id=864691136309663834").proj())
#     key_source = (minnie.Decimation() & "n_faces>500000").proj(decimation_version='version') & (minnie.BaylorSegmentCentroid() & "multiplicity=1").proj()

    
    key_source = (minnie.Decimation().proj(decimation_version='version')  & 
                  dict(decimation_version=decimation_version,decimation_ratio=decimation_ratio)  
                  & minnie.MultiSomaProofread() & (dj.U("segment_id") & (minnie.BaylorSegmentCentroid() & "multiplicity=2").proj()))

    def make(self,key):
        """
        Pseudocode for process:

        1) Get the segment id from the key
        2) Get the decimated mesh
        3) Get the somas info
        4) Run the preprocessing
        5) Calculate all starter stats
        6) Save the file in a certain location
        7) Pass stats and file location to insert
        """
        whole_pass_time = time.time()
        #1) Get the segment id from the key
        segment_id = key["segment_id"]
        description = str(key['decimation_version']) + "_25"
        print(f"\n\n----- Working on {segment_id}-------")
        global_start = time.time()
        
        #2) Get the decimated mesh
        current_neuron_mesh = du.fetch_segment_id_mesh(segment_id)

        #3) Get the somas info *************************** Need to change this when actually run *******************
        somas = du.get_soma_mesh_list(segment_id) 
        print(f"somas = {somas}")
        #4) Run the preprocessing


        total_neuron_process_time = time.time()

        print(f"\n--- Beginning preprocessing of {segment_id}---")
        recovered_neuron = neuron.Neuron(
        mesh = current_neuron_mesh,
        somas = somas,
        segment_id=segment_id,
        description=description,
        suppress_preprocessing_print=False,
        suppress_output=False,
        calculate_spines=True,
        widths_to_calculate=["no_spine_median_mesh_center"]

                )

        print(f"\n\n\n---- Total preprocessing time = {time.time() - total_neuron_process_time}")


        #5) Don't have to do any of the processing anymore because will do in the neuron object
        stats_dict = recovered_neuron.neuron_stats()



        #6) Save the file in a certain location
        save_time = time.time()
        ret_file_path = recovered_neuron.save_compressed_neuron(output_folder=str(du.get_decomposition_path()),
                                          return_file_path=True,
                                         export_mesh=False,
                                         suppress_output=True)

        ret_file_path_str = str(ret_file_path.absolute()) + ".pbz2"
        print(f"Save time = {time.time() - save_time}")



        #7) Pass stats and file location to insert
        new_key = dict(key,
                       decomposition=ret_file_path_str,
                       n_vertices=len(current_neuron_mesh.vertices),
                       n_faces=len(current_neuron_mesh.faces),
                       run_time=np.round(time.time() - whole_pass_time,4)
                      )
        new_key.update(stats_dict)

        self.insert1(new_key, allow_direct_insert=True, skip_duplicates=True)

        print(f"\n\n ------ Total time for {segment_id} = {time.time() - global_start} ------")
    

# Running the Populate

In [31]:
minnie.Decomposition()

segment_id segment id unique within each Segmentation,decimation_version,decimation_ratio ratio of remaining mesh vertices/faces (which ones depends on what metric the decimation technique uses),decomposition,n_vertices number of vertices,n_faces number of faces,n_error_limbs the number of limbs that are touching multiple somas or 1 soma in multiple places,n_same_soma_multi_touching_limbs number of limbs that touch the same soma multiple times,n_multi_soma_touching_limbs number of limbs that touch multiple somas,n_somas number of soma meshes detected,n_limbs,n_branches,max_limb_n_branches,skeletal_length,max_limb_skeletal_length,median_branch_length gives information on average skeletal length to next branch point,width_median median width from mesh center without spines removed,width_no_spine_median median width from mesh center with spines removed,width_90_perc 90th percentile for width without spines removed,width_no_spine_90_perc 90th percentile for width with spines removed,n_spines,spine_density n_spines/ skeletal_length,spines_per_branch,skeletal_length_eligible the skeletal length for all branches searched for spines,n_spine_eligible_branches the number of branches that were checked for spines because passed width threshold,spine_density_eligible n_spines/skeletal_length_eligible,spines_per_branch_eligible n_spines/n_spine_eligible_branches,total_spine_volume the sum of all spine volume,spine_volume_median median of the spine volume for those spines with able to calculate volume,spine_volume_density total_spine_volume/skeletal_length,spine_volume_density_eligible total_spine_volume/skeletal_length_eligible,spine_volume_per_branch_eligible total_spine_volume/n_spine_eligible_branches,run_time the amount of time to run (seconds)
864691134988472442,0,0.25,=BLOB=,760380,1540479,2,2,1,2,13,356,154,8945500.825073855,4539172.699946116,14745.596,83.832,83.832,450.315,368.511,2742,0.0003065228044375438,7.702247191011236,3993676.922,122,0.000686585333153797,22.475,142839986812.13495,29602421.88674418,15967.802094630699,35766.535351237646,1170819564.033893,2914.3901
864691135081644023,0,0.25,=BLOB=,853714,1717788,3,2,2,2,12,274,133,9151050.956985964,3241470.4075203836,19951.921,149.01,148.057,441.244,416.737,3461,0.0003782079256544685,12.63138686131387,6071207.457,136,0.0005700678200362805,25.449,249095068700.38538,44005779.70668538,27220.37827908988,41028.917305927826,1831581387.5028336,3155.4383
864691135340993989,0,0.25,=BLOB=,762896,1554949,1,0,1,2,11,208,69,7142831.062345771,2595026.695000718,18897.718,289.57,268.923,491.15,443.156,3109,0.00043526158925827595,14.947115384615385,6378096.19,170,0.00048744953155057384,18.288,218087987547.58118,33455405.46249962,30532.429738854706,34193.27351781146,1282870514.9857717,2687.6115
864691135538126706,0,0.25,=BLOB=,1092895,2187652,2,0,2,2,13,376,140,12979688.455596957,4108735.6392066963,16290.667,73.887,73.887,451.209,433.458,3599,0.0002772793824992062,9.571808510638299,7425884.215,140,0.0004846560888641596,25.707,202454295489.94098,26065112.000006516,15597.777726524775,27263.32509750032,1446102110.6424356,5783.7
864691135544400936,0,0.25,=BLOB=,834322,1678635,2,0,2,2,14,220,65,8654307.688300231,2071704.7158462408,22012.525,193.175,192.731,478.659,448.933,3397,0.00039252128793530283,15.440909090909091,6715055.654,123,0.0005058781602169577,27.618,116429375386.91127,19262151.443336487,13453.343650389537,17338.557025593233,946580287.6984656,3424.9471
864691135548490436,0,0.25,=BLOB=,922187,1837015,1,0,1,2,15,371,103,10075222.86068977,2764462.4406251293,14067.473,95.447,95.447,390.011,327.776,3714,0.00036862708163913827,10.01078167115903,6211914.903,151,0.0005978832707779609,24.596,223705697521.19244,33268546.01116689,22203.548309985184,36012.35706129126,1481494685.5708108,3150.0633
864691135560502753,0,0.25,=BLOB=,1480930,3012911,1,0,1,2,14,414,133,13474172.314473672,4479173.546152499,19183.266,255.669,231.318,546.743,533.845,5613,0.0004165747527193662,13.557971014492754,10369113.924,245

In [44]:
dj.config["display.limit"] = 45

INFO - 2021-01-04 22:45:31,549 - settings - Setting display.limit to 45


In [45]:
curr_table = (minnie.schema.jobs & "table_name='__decomposition'")
(curr_table & "status='error'")
#curr_table.delete()
#(curr_table & "error_message = 'ValueError: need at least one array to concatenate'").delete()

table_name className of the table,key_hash key hash,"status if tuple is missing, the job is available",key structure containing the key,error_message error message returned if failed,error_stack error stack if failed,user database user,host system hostname,pid system process id,connection_id connection_id(),timestamp automatic timestamp
__decomposition,090c7be9245f8e5aeb0e40175d2cec86,error,=BLOB=,ValueError: need at least one array to concatenate,=BLOB=,celiib@10.28.0.152,at-node25,29,21682,2021-01-04 15:23:51
__decomposition,0a6b2a3b51314bb8599539a4ed04f4a4,error,=BLOB=,ValueError: need at least one array to concatenate,=BLOB=,celiib@10.28.0.143,at-node16,29,21669,2021-01-04 13:34:00
__decomposition,0b551f0304ab0afc29b5743c69c54930,error,=BLOB=,ValueError: need at least one array to concatenate,=BLOB=,celiib@10.28.0.138,at-node11,30,21685,2021-01-04 14:46:52
__decomposition,1622bc6d61dd93b996d63e9c981f4462,error,=BLOB=,ValueError: need at least one array to concatenate,=BLOB=,celiib@10.28.0.157,at-node30,29,21674,2021-01-04 14:54:10
__decomposition,18720205cf210f8825988e024e22e1b0,error,=BLOB=,ValueError: need at least one array to concatenate,=BLOB=,celiib@10.28.0.153,at-node26,29,21663,2021-01-04 15:19:58
__decomposition,20d78a03baa098dde381e741786ffb53,error,=BLOB=,ValueError: need at least one array to concatenate,=BLOB=,celiib@10.28.0.168,at-node41,29,21659,2021-01-04 13:47:56
__decomposition,26f6cebdc9e967351f8d399546dcc588,error,=BLOB=,Exception: Something went wrong when trying to find end nodes,=BLOB=,celiib@10.28.0.157,at-node30,29,21099,2021-01-03 22:51:52
__decomposition,2c02350e8b3a39f9722dc2281dee85e4,error,=BLOB=,ValueError: need at least one array to concatenate,=BLOB=,celiib@10.28.0.143,at-node16,29,21669,2021-01-04 13:53:09
__decomposition,2d038cec8d302b165a416ce229cc8dc3,error,=BLOB=,ValueError: need at least one array to concatenate,=BLOB=,celiib@10.28.0.166,at-node39,29,21676,2021-01-04 14:34:42
__decomposition,2de78852919e90bdf37643f7564fb36b,error,=BLOB=,ValueError: need at least one array to concatenate,=BLOB=,celiib@10.28.0.137,at-node10,29,21721,2021-01-04 14:14:53


In [22]:
curr_table.delete()

In [46]:
possible_keys = [
"dd46cdbc9b22b05e1a6b60e456cf93f6",
"26f6cebdc9e967351f8d399546dcc588",
"0346046686f7bf941663a32a3d275407"
]

In [47]:
cur_error = (curr_table & dict(key_hash = possible_keys[0]))
cur_error

table_name className of the table,key_hash key hash,"status if tuple is missing, the job is available",key structure containing the key,error_message error message returned if failed,error_stack error stack if failed,user database user,host system hostname,pid system process id,connection_id connection_id(),timestamp automatic timestamp
__decomposition,dd46cdbc9b22b05e1a6b60e456cf93f6,error,=BLOB=,"Exception: The number of nodes in the concept graph and number of branches passed to it did not match len(curr_limb_concept_network.nodes())=88, len(curr_limb_divided_skeletons)= 99",=BLOB=,celiib@10.28.0.81,at-compute003,29,21094,2021-01-03 23:20:35


In [49]:
print(cur_error.fetch1("error_stack"))

Traceback (most recent call last):
  File "/usr/local/lib/python3.6/dist-packages/datajoint/autopopulate.py", line 159, in populate
    make(dict(key))
  File "Decomposition_for_multi_soma.py", line 223, in make
    widths_to_calculate=["no_spine_median_mesh_center"]
  File "/meshAfterParty/neuron.py", line 1745, in __init__
    use_meshafterparty=use_meshafterparty)
  File "/meshAfterParty/preprocessing_vp2.py", line 3088, in preprocess_neuron
    run_concept_network_checks=True,
  File "/meshAfterParty/preprocessing_vp2.py", line 730, in calculate_limb_concept_networks
    raise Exception("The number of nodes in the concept graph and number of branches passed to it did not match\n"
Exception: The number of nodes in the concept graph and number of branches passed to it did not match
len(curr_limb_concept_network.nodes())=88, len(curr_limb_divided_skeletons)= 99



In [ ]:
# import pandas as pd
# key_hash,error_message = curr_table.fetch("key_hash","error_message")

# df = pd.DataFrame.from_dict([dict(key_hash = k,error_message = m) for k,m in zip(key_hash,error_message)])
# df
# #df.columns = ["error","key_hash"]
# key_hashes_to_delete = df[df["error_message"].str.contains("OSError")]["key_hash"].to_numpy()

# (curr_table & [dict(key_hash=k) for k in key_hashes_to_delete]).delete()

In [ ]:
import time
import random
import compartment_utils as cu
cu = reload(cu)
import preprocessing_vp2 as pre
pre = reload(pre)
import skeleton_utils as sk
sk = reload(sk)
nru = reload(nru)

start_time = time.time()
if not test_mode:
    time.sleep(random.randint(0, 800))
print('Populate Started')
if not test_mode:
    Decomposition.populate(reserve_jobs=True, suppress_errors=True)
else:
    Decomposition.populate(reserve_jobs=True, suppress_errors=False)
print('Populate Done')

print(f"Total time for DecompositionMultiSoma populate = {time.time() - start_time}")

In [ ]:
debug